<img src="./ESSENCE.png" width=700px>

### Reflection pattern

## input -> function(input) -> output -> function(output) -> output2

<img src="./INPUT_OUTPUT.png" width=700px>




## Agent One is a code expert. Agent Two is an expert code reviewer.

We generate a response with our first query using a system prompt to create code.

We then pass the output into another function that acts as a reviewer to produce the next version of the code.

This can be repeated until we reach certain criteria or MAX_ITERATIONS, whichever comes first.

Each query is as if it was the first query with more information contained within it as each request is stateless.

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display_markdown

In [2]:
client = OpenAI()
MODEL = "gpt-4o-mini"

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:14]}...")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-TVyeca...


We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python
programmer eager to receive feedback / critique by the user.


In [4]:
content = """
You are a Python programmer tasked with generating high quality Python code. Your task is to Generate the best content possible for the user's request. If the user requests critique, respond with a revised version of your previous attempt.
"""

generation_chat_history = [{"role": "system", "content": content}]

Now, as the user, we are going to ask the LLM to generate an implementation of the Requests library


In [5]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of requesting an API with the Requests library",
    }
)

Let's generate the first version of the essay.


In [6]:
user_code = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)


generation_chat_history.append({"role": "assistant", "content": user_code})

In [7]:
display_markdown(user_code, raw=True)
# Output below...

Here's a simple Python implementation that demonstrates how to request data from an API using the Requests library. This example includes error handling and JSON response parsing.

```python
import requests

def fetch_data_from_api(url):
    try:
        # Send a GET request to the specified URL
        response = requests.get(url)
        
        # Raise an exception for HTTP error responses
        response.raise_for_status()

        # Parse JSON response if content is in JSON format
        if response.headers['Content-Type'] == 'application/json':
            data = response.json()
            return data
        
        # Return response text if not JSON
        return response.text
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with a valid API endpoint
    result = fetch_data_from_api(api_url)
    
    if result is not None:
        print("Retrieved data:", result)
    else:
        print("Failed to retrieve data.")
```

### Explanation:
1. **Importing Requests Library**: The `requests` module is imported to handle HTTP requests.
2. **Function `fetch_data_from_api`**: This function takes a URL as an argument:
   - It sends a GET request to that URL.
   - It raises exceptions for any HTTP errors (like 404 or 500).
   - It checks if the response is in JSON format and parses it accordingly.
   - If the response isn't JSON, it returns the response text.
3. **Error Handling**: The `try-except` block captures any exceptions that may arise during the request, making it more robust.
4. **Example Usage**: The script includes an example of how to call the function with a sample API URL.

Make sure to install the Requests library if you haven't already by using:
```bash
pip install requests
```

Feel free to replace the example URL with any valid API endpoint to test the code!

## Reflection Step


This is equivalent to asking a follow up question in say ChatGPT and we change the system prompt or what we want it to do along with the output from the previous query.


In [8]:
reflection_chat_history = [
    {
        "role": "system",
        "content": "You are an experienced and talented Pythonista. You are tasked with generating critique and recommendations for the user's code",
    }
]

In [9]:
# We add new messages to the list of messages so that the LLM has context and knowledge of what proceeded.

# LLM calls are stateless and previous messages are not stored with the LLM. This is an important fact as we do not want to go over the context window for the LLM or incur unwanted costs if applicable.

reflection_chat_history.append({"role": "user", "content": user_code})

CRITIQUE TIME


Now that we have the context and the request for a critique, we make a request to the LLM.


In [10]:
critique = (
    client.chat.completions.create(messages=reflection_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [11]:
# Display

display_markdown(critique, raw=True)
# Critique displayed below...

Your implementation for fetching data from an API using the `requests` library in Python is quite solid. It effectively demonstrates essential practices, such as error handling and response parsing. Below are some critiques and recommendations for potential improvements:

### Critiques:

1. **Content-Type Check**:
   - You're currently checking the `Content-Type` header directly from the response. While this often works, it's best practice to use the `response.json()` method within a try-except block to handle cases where the response might not be in JSON format, which will automatically raise a `JSONDecodeError`.

2. **Handling Non-200 Status Codes**:
   - The `raise_for_status()` method effectively handles non-200 status codes. However, you could provide additional context in the exception message for better debugging.

3. **Return Value Consistency**:
   - The function may return either a JSON object (if the content is JSON) or a string (if it's not JSON). This could potentially lead to confusion since the return type is not consistent. Documenting this in your function's docstring may help users understand the expected behavior.

4. **Retries for API Requests**:
   - To handle transient issues, consider adding a retry mechanism using `requests` built-in support or the `tenacity` library. This can improve robustness further, particularly in scenarios where network issues are common.

5. **Logging Instead of Printing**:
   - Instead of using print statements for errors, consider using the `logging` library, which provides more flexibility and control over how messages are logged.

6. **Docstring**:
   - Adding a docstring to explain the function’s purpose, parameters, return, and any exceptions that may be raised will enhance code readability and maintainability.

### Recommendations:

Here’s an enhanced version of your code based on the critiques and recommendations above:

```python
import requests
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url):
    """
    Fetch data from the specified API URL.
    
    Args:
        url (str): The URL of the API endpoint.
        
    Returns:
        dict or str: Parsed JSON data if the response is JSON, 
                     else the raw response text. None if an error occurs.
    """
    try:
        # Send a GET request to the specified URL
        response = requests.get(url)
        
        # Raise an exception for HTTP error responses
        response.raise_for_status() 

        # Attempt to parse JSON response
        try:
            return response.json()
        except ValueError:
            logging.warning(f"Response content is not JSON. Returning text instead.")
            return response.text
        
    except requests.exceptions.RequestException as e:
        logging.error(f"An error occurred: {e}")
        return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with a valid API endpoint
    result = fetch_data_from_api(api_url)
    
    if result is not None:
        logging.info("Retrieved data: %s", result)
    else:
        logging.error("Failed to retrieve data.")
```

### Key Improvements:
- **Error Logging**: This code utilizes the `logging` library for clearer and more adjustable error handling.
- **Improved JSON Handling**: It attempts JSON parsing separately and catches any errors, allowing for better handling if the response isn't JSON.
- **Docstring Addition**: Gives clarity on the function's parameters and return types, improving maintainability.

These enhancements should create a more robust and clear implementation for users interacting with your API-fetching code.

Add CRITIQUE to chat...

Notice how we are appending previous responses so that the next SYSTEM MESSAGE has history or context.


In [12]:
generation_chat_history.append({"role": "user", "content": critique})

Response to CRITIQUE


In [13]:
essay = (
    client.chat.completions.create(messages=generation_chat_history, model=MODEL)
    .choices[0]
    .message.content
)

In [14]:
# Diaply user response to CRITIQUE
display_markdown(essay, raw=True)
# Response to critique displayed below...

Here's the revised version of the API fetching implementation based on your critiques and suggestions:

```python
import requests
import logging

# Set up logging configuration
logging.basicConfig(level=logging.INFO)

def fetch_data_from_api(url):
    """
    Fetch data from the specified API URL.

    Args:
        url (str): The URL of the API endpoint.

    Returns:
        dict or str: Parsed JSON data if the response is JSON,
                     else the raw response text. Returns None if an error occurs.
    """
    try:
        # Send a GET request to the specified URL
        response = requests.get(url)
        
        # Raise an exception for HTTP error responses
        response.raise_for_status()

        # Attempt to parse JSON response
        try:
            return response.json()
        except ValueError:
            logging.warning("Response content is not JSON. Returning text instead.")
            return response.text
        
    except requests.exceptions.RequestException as e:
        logging.error("An error occurred while fetching data: %s", e)
        return None

# Example usage
if __name__ == "__main__":
    api_url = "https://api.example.com/data"  # Replace with a valid API endpoint
    result = fetch_data_from_api(api_url)
    
    if result is not None:
        logging.info("Retrieved data: %s", result)
    else:
        logging.error("Failed to retrieve data.")
```

### Key Improvements:
1. **Error Handling**: The function uses a separate `try-except` block around the JSON parsing to catch `ValueError`, which provides a clear distinction when dealing with non-JSON responses.
2. **Logging**: `logging` has been used instead of `print`, enabling better control over the output and the ability to set different logging levels.
3. **Consistent Return Documentation**: The docstring now includes a clear explanation of what the function returns based on the API response.

This version of the code is more robust and user-friendly, improving the maintainability and clarity for future modifications or inquiries. Thank you for your feedback!

<br>

### We can of course make this a Class...
